<a href="https://colab.research.google.com/github/gurmeetheera/Latent-Dirichlet-Allocation/blob/main/trexquantHangman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [80]:
HANGMAN_URL = "https://sg.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        alphabets = "abcdefghijklmnopqrstuvwxyz"
        uniGram = {}
        biGram = {}
        triGram = {}
        for dict_word in current_dictionary:
          for albt in dict_word:
            uniGram[albt] = uniGram.get(albt, 0) + 1

        for dict_word in current_dictionary:
          for i in range(len(dict_word) - 1):
            pair = dict_word[i: i+2]
            biGram[pair] = biGram.get(pair, 0) + 1

        for dict_word in current_dictionary:
          for i in range(len(dict_word) - 2):
            trigram = dict_word[i: i+3]
            triGram[trigram] = triGram.get(trigram, 0) + 1




        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'

        # for i in range(len(clean_word) - 3):
        #   tg = hh[i:i + 4]
        #   missingalphabets = tg.count(".")
        #   srt_a = []
        #   srt_c = []
        #   if missingalphabets == 1:
        #     matchcounts = [triGram.get(tg.replace(".", x),0) for x in alphabets]
        #     srt_a.append(alphabets[matchcounts.index(max(matchcounts))])
        #     srt_c.append(max(matchcounts))
        #     letter = srt_a[srt_c.index(max(srt_c))]
        #     if letter not in self.guessed_letters:
        #         guess_letter = letter
        #         break


        # for i in range(len(clean_word) - 2):
        #   tg = hh[i:i+3]
        #   missingalphabets = tg.count(".")
        #   srt_a = []
        #   srt_c = []
        #   if missingalphabets == 1:
        #     matchcounts = [triGram.get(tg.replace(".", x),0) for x in alphabets]
        #     srt_a.append(alphabets[matchcounts.index(max(matchcounts))])
        #     srt_c.append(max(matchcounts))
        #     letter = srt_a[srt_c.index(max(srt_c))]
        #     if letter not in self.guessed_letters:
        #         guess_letter = letter
        #         break

        for i in range(len(clean_word) - 1):
          tg = hh[i:i + 2]
          missingalphabets = tg.count(".")
          srt_a = []
          srt_c = []
          if missingalphabets == 1:
            matchcounts = [triGram.get(tg.replace(".", x),0) for x in alphabets]
            srt_a.append(alphabets[matchcounts.index(max(matchcounts))])
            srt_c.append(max(matchcounts))
            letter = srt_a[srt_c.index(max(srt_c))]
            if letter not in self.guessed_letters:
                guess_letter = letter
                break

        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    HANGMAN_URL + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [81]:
api = HangmanAPI(access_token="4ceb2aff1465bfb20da9d913028bf6", timeout=2000)


In [ ]:
for i in range(100):
  api.start_game(practice=1,verbose=False)


In [83]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: b266512a3710. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ i _ _ i _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ i _ _ i _ _ _ '}
Guessing letter: l
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ i _ _ i _ _ l '}
Guessing letter: a
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ i _ _ i _ a l '}
Guessing letter: t
Sever response: {'game_id': 'b266512a3710', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ i _ _ i _ a l '}
Guessing letter: c
Sever response: {'game_id': 'b266512a3710', 

In [7]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: ignored

In [8]:
hh = ".pp.e"


In [12]:
def build_dictionary(dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

In [23]:
full_dictionary = build_dictionary("words_250000_train.txt")

In [24]:
type(full_dictionary)

list

In [34]:
alphabets = "abcdefghijklmnopqrstuvwxyz"
uniGram = {}
biGram = {}
triGram = {}
for dict_word in full_dictionary:
  for albt in dict_word:
    uniGram[albt] = uniGram.get(albt, 0) + 1

for dict_word in full_dictionary:
  for i in range(len(dict_word) - 1):
    pair = dict_word[i: i+2]
    biGram[pair] = biGram.get(pair, 0) + 1

for dict_word in full_dictionary:
  for i in range(len(dict_word) - 2):
    trigram = dict_word[i: i+3]
    triGram[trigram] = triGram.get(trigram, 0) + 1

In [52]:
for i in range(len(hh) - 2):
  tg = hh[i:i+3]
  print(tg)
  missingalphabets = tg.count(".")
  srt_a = []
  srt_c = []
  if missingalphabets == 1:
    matchcounts = [triGram.get(tg.replace(".", x),0) for x in alphabets]
    srt_a.append(alphabets[matchcounts.index(max(matchcounts))])
    srt_c.append(max(matchcounts))
    guess = srt_a[srt_c.index(max(srt_c))]

.pp
pp.
p.e


In [54]:
for i in range(len(hh) - 1):
  tg = hh[i:i + 2]
  print(tg)
  missingalphabets = tg.count(".")
  srt_a = []
  srt_c = []
  if missingalphabets == 1:
    matchcounts = [triGram.get(tg.replace(".", x),0) for x in alphabets]
    srt_a.append(alphabets[matchcounts.index(max(matchcounts))])
    srt_c.append(max(matchcounts))
    guess = srt_a[srt_c.index(max(srt_c))]

.p
pp
p.
.e


In [55]:
guess

'a'

In [10]:
hh.replace(".","z")

'zppze'